##### Main authors: WENQI HOU, GAURAVI SAHA, MANYING (JANE) TSANG
#### Repurposed with adaptations and changes from: GIOVANNI FICARRA & LEONARDO PICCHIAMI

### YELP DATA PREPROCESSING - RESTAURANT RECOMMENDATION SYSTEM 

### 1. Description of Data

5 datasets in json format retrieved from Yelp website : business.json, user.json, checkin.json, tip.json and review.json.

#### business dataset

- business_id: ID of the business
- name: name of the business
- neighborhood
- address: address of the business
- city: city of the business
- state: state of the business
- postal_code: postal code of the business
- latitude: latitude of the business
- longitude: longitude of the business
- stars: average rating of the business
- review_count: number of reviews received
- is_open: 1 if the business is open, 0 therwise
- categories: multiple categories of the business


#### review dataset

- review_id: ID of the review
- user_id: ID of the user
- business_id: ID of the business
- stars: ratings of the business
- date: review date
- text: review from the user
- useful: number of users who vote a review as usefull
- funny: number of users who vote a review as funny
- cool: number of users who vote a review as cool


#### user dataset

- average stars
- compliment_cool, compliment_cute, compliment_funny, compliment_hot, compliment_list, compliment_more, compliment_note, compliment_photos, compliment_plain, compliment_profile, compliment_writer
- cool
- elite
- fans
- friends
- funny
- name
- review_counts
- useful
- user_id
- yelping_since


#### checkin dataset 

- business_id
- date


#### tip dataset

- business_id
- compliment_count
- date
- text
- user_id

### 2. Data Processing Tasks

#### Generating a cleaned and transformed version of the data:

1. Transfer json into pandas dataframe with proper indexing Extract data that includes all restaurants.
2. Replace garbage data which includes incorrect states and postal codes, etc Replace missing values. 
3. Date transformations and standardization.
4. Merge multiple dataframes and reshape.
5. Create new explanatory features based on initial features.
6. Delete conseguently unnecessary columns which could add ambiguity based on new features.
7. Delete duplicate restaurants entries and combine their reviews.


### Enhancement to the Data:

We have improved and enhanced the data at every level by cleaning information within the columns. Further data cleaning and enhancements are covered in the data cleaning section.

### 3. Explanatory Data Analysis and Features Engineering

### Data Import

We imported our large json file into dataframes by spliting each file into multiple chunks, then convert these chunks to a list, and concatenated them to a final dataframe.
After creating one dataframe, we check the columns, the shapes and the head of the dataframe to get an overall idea of what our data looks like and its features.

In [6]:
import warnings
warnings.filterwarnings("ignore")

import re
from collections import Counter
import datetime as dt

import pandas as pd
import numpy as np
import pickle

In [7]:
frames_tip = []
for chunk in pd.read_json('../dataset/yelp_academic_dataset_tip.json', lines=True, chunksize = 10000):
    frames_tip.append(chunk)
tip=pd.concat(frames_tip)

In [8]:
tip.columns

Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')

In [9]:
tip.head()

,user_id,business_id,text,date,compliment_count
0,UPw5DWs_b-e2JRBS-t37Ag,VaKXUpmWTTWDKbpJ3aQdMw,"Great for watching games, ufc, and whatever el...",2014-03-27 03:51:24,0
1,Ocha4kZBHb4JK0lOWvE0sg,OPiPeoJiv92rENwbq76orA,Happy Hour 2-4 daily with 1/2 price drinks and...,2013-05-25 06:00:56,0
2,jRyO2V1pA4CdVVqCIOPc1Q,5KheTjYPu1HcQzQFtm4_vw,Good chips and salsa. Loud at times. Good serv...,2011-12-26 01:46:17,0
3,FuTJWFYm4UKqewaosss1KA,TkoyGi8J7YFjA6SbaRzrxg,The setting and decoration here is amazing. Co...,2014-03-23 21:32:49,0
4,LUlKtaM3nXd-E4N4uOk_fQ,AkL6Ous6A1atZejfZXn1Bg,Molly is definately taking a picture with Sant...,2012-10-06 00:19:27,0


In [10]:
frames_checkin = []
for chunk in pd.read_json('../dataset/yelp_academic_dataset_checkin.json', lines=True, chunksize = 10000):
    frames_checkin.append(chunk)
checkin=pd.concat(frames_checkin)

In [11]:
checkin.columns

Index(['business_id', 'date'], dtype='object')

In [12]:
checkin.shape

(161950, 2)

In [13]:
checkin.head()

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."


In [14]:
frames_review = []
for chunk in pd.read_json('../dataset/yelp_academic_dataset_review.json', lines=True, chunksize = 10000):
    frames_review.append(chunk)
review=pd.concat(frames_review)

In [15]:
review.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [16]:
review.shape

(6685900, 9)

In [17]:
review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [18]:
frames = []
for chunk in pd.read_json('../dataset/yelp_academic_dataset_user.json', lines=True, chunksize = 10000):
    frames.append(chunk)
user = pd.concat(frames)

In [19]:
user.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [20]:
user.shape

(1637138, 22)

In [21]:
user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,84,17,25,"2015,2016,2017","c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",5,...,0,0,0,0,1,1,1,1,2,0
1,4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,48,22,16,,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",4,...,0,0,0,0,0,0,1,1,0,0
2,bc8C_eETBWL0olvFSJJd0w,David,16,2013-10-04 00:16:10,28,8,10,,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",0,...,0,0,0,0,1,0,0,0,0,0
3,dD0gZpBctWGdWo9WlGuhlA,Angela,17,2014-05-22 15:57:30,30,4,14,,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",5,...,0,0,0,0,0,2,0,0,1,0
4,MM4RJAeH6yuaN8oZDSt0RA,Nancy,361,2013-10-23 07:02:50,1114,279,665,"2015,2016,2017,2018","mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",39,...,1,0,0,1,16,57,80,80,25,5


In [22]:
frames_business = []
for chunk in pd.read_json('../dataset/yelp_academic_dataset_business.json', lines=True, chunksize = 10000):
    frames_business.append(chunk)
business = pd.concat(frames_business)

In [23]:
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [24]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


### Flow of Data Processing:

We started with 'business' since it contains ‘attribute’ which we can use it to extract all business for all cities, and further extract restaurants based on ‘categories’ out of all business types. 

- By creating a new dataframe business, we were able to filter 'review' table by matching its 'business_id' with 'business_id' in dataframe'business', creating a new dataframe 'all_reviews'.

- Using the same logic, we then were able to filter 'all_users' dataframe by matching its 'user_id' with 'user_id' in 'all_review'. 

- The new dataframe 'all_review'contains all customers who have been to at least one restaurant and left a review. 

- Same as the rest two dataframes, new dataframes 'all_tips', 'all_checkin' were created by matching 'business_id'

- Saving all dataframes as pickle files.

#### To avoid importing data from the large json files every time, we converted the new dataframes to pickle files for future use.

In [25]:
business['restaurant'] = business['categories'].str.contains('Restaurants',flags = re.IGNORECASE)

In [26]:
business_restaurant = business[business['restaurant'] == True]

In [27]:
business_restaurant.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,restaurant
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",True
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",True
11,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",True
13,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,5981 Andrews Rd,Mentor-on-the-Lake,OH,44060,41.708520,-81.359556,4.0,16,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Italian, Restaurants, Pizza, Chicken Wings","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",True
17,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119,36.100016,-115.128529,4.0,40,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...","Restaurants, Italian",None,True


In [28]:
business_restaurant.reset_index(drop=True).head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,restaurant
0,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",True
1,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",True
2,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",True
3,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,5981 Andrews Rd,Mentor-on-the-Lake,OH,44060,41.708520,-81.359556,4.0,16,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Italian, Restaurants, Pizza, Chicken Wings","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",True
4,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119,36.100016,-115.128529,4.0,40,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...","Restaurants, Italian",None,True


In [29]:
business_restaurant.shape

(59371, 15)

#### The Pickling Process

In [30]:
business_restaurant.to_pickle('../dataset/restaurants.pickle')

In [31]:
review_all_restaurant = review.loc[review['business_id'].isin(business_restaurant['business_id'].unique())]

In [32]:
review_all_restaurant.reset_index(drop=True).head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
1,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,4,0,0,0,I'll be the first to admit that I was not exci...,2013-01-20 13:25:59
2,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3,5,4,5,Tracy dessert had a big name in Hong Kong and ...,2016-05-07 01:21:02
3,8e9HxxLjjqc9ez5ezzN7iQ,d6xvYpyzcfbF_AZ8vMB7QA,zvO-PJCpNk4fgAVUnExYAA,1,3,1,1,This place has gone down hill. Clearly they h...,2010-10-05 19:12:35
4,kbtscdyz6lvrtGjD1quQTg,FIk4lQQu1eTe2EpzQ4xhBA,8mIrX_LrOnAqWsB5JrOojQ,4,0,0,0,"Like walking back in time, every Saturday morn...",2011-11-30 02:11:15


In [33]:
review_all_restaurant.to_pickle('../dataset/all_review.pickle')

In [34]:
user.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [35]:
user_all_restaurant=user.loc[user['user_id'].isin(review_all_restaurant['user_id'].unique())]

In [36]:
user_all_restaurant.to_pickle('../dataset/all_users.pickle')

In [37]:
tip.columns

Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')

In [38]:
tip_all_restaurant = tip.loc[tip['user_id'].isin(review_all_restaurant['user_id'].unique())].reset_index(drop=True)

In [39]:
tip_all_restaurant.to_pickle('../dataset/all_tips.pickle')

In [40]:
check_all_restaurant = checkin.loc[checkin['business_id'].isin(business_restaurant['business_id'].unique())].reset_index(drop=True)

In [41]:
check_all_restaurant.to_pickle('../dataset/all_checkin.pickle')

# Data Cleaning - Making the Data useful for analysis

##### Working with Business pickle file:

This file contains information about our restaurants and other related parameters. This dataframe acts as the focus of our analysis and we intend to derive meaningful insights from it.

Summary of actions:
- Reading the business pickle file for clean up
- Missing values cleaned up
- Using only a few selected columns for meaningful analysis
- Extract useful information from categories column to investigate resturants' cuisine


In [42]:
rest = pd.read_pickle('../dataset/restaurants.pickle')

In [43]:
rest.fillna(value = pd.np.nan, inplace = True)

In [44]:
Rest = rest.reset_index(drop = True)
Rest.index += 1
Rest.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,restaurant
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",True
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",True
3,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",True
4,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,5981 Andrews Rd,Mentor-on-the-Lake,OH,44060,41.708520,-81.359556,4.0,16,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Italian, Restaurants, Pizza, Chicken Wings","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",True
5,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119,36.100016,-115.128529,4.0,40,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...","Restaurants, Italian",NaN,True


In [45]:
Rest.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'restaurant'],
      dtype='object')

In [46]:
#Adding the feature city among those already considered
Rest_final = Rest[['name', 'business_id', 'address', 'categories', 'postal_code','attributes','hours','latitude','longitude','review_count','stars', 'city']]

In [47]:
categories=', '.join(list(Rest_final['categories'].unique()))
categories=categories.split(', ')
categories[:5]

['Specialty Food', 'Restaurants', 'Dim Sum', 'Imported Food', 'Food']

In [48]:
c = Counter(categories)
c.most_common(60)

[('Restaurants', 31076),
 ('Food', 12519),
 ('Nightlife', 7971),
 ('Bars', 7606),
 ('American (Traditional)', 5301),
 ('Sandwiches', 4848),
 ('Breakfast & Brunch', 4195),
 ('American (New)', 3925),
 ('Burgers', 3069),
 ('Pizza', 2921),
 ('Fast Food', 2907),
 ('Coffee & Tea', 2644),
 ('Event Planning & Services', 2575),
 ('Cafes', 2428),
 ('Italian', 2392),
 ('Salad', 2255),
 ('Specialty Food', 2062),
 ('Seafood', 1942),
 ('Chinese', 1826),
 ('Chicken Wings', 1826),
 ('Mexican', 1753),
 ('Caterers', 1727),
 ('Sports Bars', 1707),
 ('Desserts', 1697),
 ('Bakeries', 1600),
 ('Asian Fusion', 1543),
 ('Canadian (New)', 1538),
 ('Pubs', 1440),
 ('Delis', 1396),
 ('Mediterranean', 1357),
 ('Japanese', 1293),
 ('Barbeque', 1280),
 ('Steakhouses', 1241),
 ('Beer', 1126),
 ('Wine & Spirits', 1126),
 ('Sushi Bars', 1072),
 ('Vegetarian', 1042),
 ('Arts & Entertainment', 1034),
 ('Cocktail Bars', 962),
 ('Middle Eastern', 944),
 ('Wine Bars', 940),
 ('Diners', 940),
 ('Lounges', 896),
 ('Juice Bar

In [49]:
cuisine = 'American|Chinese|Italian|Japanese|Mexican|Asian Fusion|Thai|Korean|Mediterranean'
Rest_final['cuisine']=Rest_final['categories'].str.findall(cuisine)

In [50]:
Rest_final['cuisine'] = Rest_final['cuisine'].map(lambda x: list(x))
Rest_final['cuisine'] = Rest_final['cuisine'].map(lambda x: ['Others'] if x==[] else x)

In [51]:
Rest_final['cuisine'].head(20)

1                         [Chinese]
2                        [Japanese]
3                         [Mexican]
4                         [Italian]
5                         [Italian]
6                          [Others]
7                         [Mexican]
8                          [Others]
9                          [Others]
10    [Italian, American, American]
11                        [Chinese]
12                         [Others]
13                         [Others]
14                         [Others]
15                         [Others]
16                         [Korean]
17                       [Japanese]
18                         [Others]
19                       [American]
20                        [Chinese]
Name: cuisine, dtype: object

#### Remove redundant entries (e.g: American, American)

In [52]:
Rest_final['cuisine'] = Rest_final['cuisine'].map(lambda x: list(dict.fromkeys(x)))
Rest_final['cuisine'] = Rest_final['cuisine'].map(', '.join) # convert list of string to string
Rest_final['cuisine'].head(20)

1               Chinese
2              Japanese
3               Mexican
4               Italian
5               Italian
6                Others
7               Mexican
8                Others
9                Others
10    Italian, American
11              Chinese
12               Others
13               Others
14               Others
15               Others
16               Korean
17             Japanese
18               Others
19             American
20              Chinese
Name: cuisine, dtype: object

Check all cuisines and merge all resturants with cuisine - Asian into Asian fusion for ease.

In [53]:
Rest_final['cuisine'].unique()

array(['Chinese', 'Japanese', 'Mexican', 'Italian', 'Others',
       'Italian, American', 'Korean', 'American', 'Italian, Japanese',
       'Mediterranean', 'Thai', 'American, Asian Fusion, Mexican, Korean',
       'Chinese, Asian Fusion',
       'Chinese, Japanese, Asian Fusion, Mexican',
       'Asian Fusion, Japanese', 'Asian Fusion, Chinese',
       'Chinese, Italian', 'Italian, Mediterranean', 'American, Mexican',
       'Mexican, American', 'American, Mediterranean',
       'Korean, Asian Fusion', 'Asian Fusion, Thai',
       'Japanese, Asian Fusion', 'Asian Fusion',
       'Asian Fusion, Japanese, Korean', 'American, Italian',
       'Japanese, American', 'Japanese, Asian Fusion, Chinese',
       'Asian Fusion, Japanese, American', 'Chinese, Thai',
       'Japanese, Korean', 'Asian Fusion, Mexican', 'Korean, Thai',
       'Thai, Chinese', 'Korean, Japanese', 'Chinese, Japanese',
       'Thai, Japanese, Chinese', 'Korean, Japanese, Chinese',
       'Chinese, Asian Fusion, Japanes

In [54]:
Rest_final['cuisine'].iloc[np.where(Rest_final['cuisine'].str.contains('Asian Fusion'))]='Asian Fusion'

In [55]:
Rest_final['cuisine'].unique()

array(['Chinese', 'Japanese', 'Mexican', 'Italian', 'Others',
       'Italian, American', 'Korean', 'American', 'Italian, Japanese',
       'Mediterranean', 'Thai', 'Asian Fusion', 'Chinese, Italian',
       'Italian, Mediterranean', 'American, Mexican', 'Mexican, American',
       'American, Mediterranean', 'American, Italian',
       'Japanese, American', 'Chinese, Thai', 'Japanese, Korean',
       'Korean, Thai', 'Thai, Chinese', 'Korean, Japanese',
       'Chinese, Japanese', 'Thai, Japanese, Chinese',
       'Korean, Japanese, Chinese', 'Japanese, Chinese', 'Thai, Japanese',
       'American, Mediterranean, Italian', 'Mediterranean, American',
       'Japanese, Thai', 'Mediterranean, Italian', 'American, Japanese',
       'American, Chinese', 'Chinese, Korean', 'American, Thai, Chinese',
       'Korean, American', 'Mediterranean, Italian, American',
       'Italian, American, Mediterranean', 'Mexican, American, Italian',
       'Italian, Thai', 'Korean, Mexican',
       'Mediterra

#### Analysis of messy data in the attribute column:

To fix this issue where each item inside is a dictionary with values, attributes acts as a filter on Yelp that customers can click to identify the restaurant. For eg. Wifi = Yes would be selected (or tick marked) while making a selection on Yelp.

We have split the atributes column with dictionary to different filters.

In [56]:
Rest_final.isnull().sum()

name                0
business_id         0
address             0
categories          0
postal_code         0
attributes       2210
hours           13995
latitude            0
longitude           0
review_count        0
stars               0
city                0
cuisine             0
dtype: int64

In [57]:
Rest_final['attributes'].apply(pd.Series).head()
# Split the attributes dictionary into all its values

,0,AcceptsInsurance,AgesAllowed,Alcohol,Ambience,BYOB,BYOBCorkage,BestNights,BikeParking,BusinessAcceptsBitcoin,...,RestaurantsCounterService,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi
1,NaN,NaN,NaN,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,NaN,...,NaN,False,True,2,True,True,True,NaN,NaN,u'no'
2,NaN,NaN,NaN,u'beer_and_wine',"{'romantic': False, 'intimate': False, 'touris...",NaN,NaN,NaN,True,NaN,...,NaN,False,True,2,True,True,True,NaN,NaN,u'no'
3,NaN,NaN,NaN,u'none',"{'romantic': False, 'intimate': False, 'touris...",NaN,NaN,NaN,NaN,NaN,...,NaN,False,True,1,False,NaN,True,NaN,NaN,u'no'
4,NaN,NaN,NaN,u'none',None,NaN,NaN,NaN,True,NaN,...,NaN,True,True,2,False,NaN,True,NaN,NaN,NaN
5,NaN,NaN,NaN,u'full_bar',"{'romantic': True, 'intimate': False, 'tourist...",NaN,'no',NaN,NaN,NaN,...,NaN,False,True,2,True,NaN,True,NaN,NaN,u'no'


#### Summary of actions:
- Concatenating the attributes to the dataframe.
- Since there are a lot of missing values in most of the columns, we have cherry-picked a few columns out of the list and included a few filters for our analysis.
- Clean up of the WiFi column.
- Clean up of the Alcohol column.

 

In [58]:
R = Rest_final['attributes'].apply(pd.Series)
list(R.columns)

[0,
 'AcceptsInsurance',
 'AgesAllowed',
 'Alcohol',
 'Ambience',
 'BYOB',
 'BYOBCorkage',
 'BestNights',
 'BikeParking',
 'BusinessAcceptsBitcoin',
 'BusinessAcceptsCreditCards',
 'BusinessParking',
 'ByAppointmentOnly',
 'Caters',
 'CoatCheck',
 'Corkage',
 'DietaryRestrictions',
 'DogsAllowed',
 'DriveThru',
 'GoodForDancing',
 'GoodForKids',
 'GoodForMeal',
 'HairSpecializesIn',
 'HappyHour',
 'HasTV',
 'Music',
 'NoiseLevel',
 'Open24Hours',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsCounterService',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'RestaurantsTakeOut',
 'Smoking',
 'WheelchairAccessible',
 'WiFi']

In [59]:
Rest_new = pd.concat([Rest_final.drop(['attributes'], axis=1), Rest_final['attributes'].apply(pd.Series)], axis=1)
Rest_new.head()

,name,business_id,address,categories,postal_code,hours,latitude,longitude,review_count,stars,...,RestaurantsCounterService,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi
1,Emerald Chinese Restaurant,QXAEGFB4oINsVuTFxEYKFQ,30 Eglinton Avenue W,"Specialty Food, Restaurants, Dim Sum, Imported...",L5R 3E7,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",43.605499,-79.652289,128,2.5,...,NaN,False,True,2,True,True,True,NaN,NaN,u'no'
2,Musashi Japanese Restaurant,gnKjwL_1w79qoiV3IC_xQQ,"10110 Johnston Rd, Ste 15","Sushi Bars, Restaurants, Japanese",28210,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",35.092564,-80.859132,170,4.0,...,NaN,False,True,2,True,True,True,NaN,NaN,u'no'
3,Taco Bell,1Dfx3zM-rW4n-31KeC8sJg,2450 E Indian School Rd,"Restaurants, Breakfast & Brunch, Mexican, Taco...",85016,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",33.495194,-112.028588,18,3.0,...,NaN,False,True,1,False,NaN,True,NaN,NaN,u'no'
4,Marco's Pizza,fweCYi8FmbJXHCqLnwuk8w,5981 Andrews Rd,"Italian, Restaurants, Pizza, Chicken Wings",44060,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",41.708520,-81.359556,16,4.0,...,NaN,True,True,2,False,NaN,True,NaN,NaN,NaN
5,Carluccio's Tivoli Gardens,PZ-LZzSlhSe9utkQYU8pFg,"1775 E Tropicana Ave, Ste 29","Restaurants, Italian",89119,NaN,36.100016,-115.128529,40,4.0,...,NaN,False,True,2,True,NaN,True,NaN,NaN,u'no'


In [60]:
Rest_new = Rest_new[['name', 'business_id', 'address', 'cuisine', 'postal_code','hours','latitude','longitude',
                   'review_count','stars','OutdoorSeating','BusinessAcceptsCreditCards','RestaurantsDelivery',
                   'RestaurantsReservations','WiFi','Alcohol','categories', 'city']]

In [61]:
Rest_new.fillna(value=pd.np.nan, inplace=True)
Rest_new['WiFi'].unique()

array(["u'no'", nan, "u'free'", "'no'", "'free'", "u'paid'", 'None',
       "'paid'"], dtype=object)

In [62]:
a=Rest_new['WiFi'].map(lambda x: 'No' if x in np.array(["u'no'", "'no'",'None']) else x)
a=a.map(lambda x: 'Free' if x in np.array(["'free'", "u'free'"]) else x)
a.unique()

array(['No', nan, 'Free', "u'paid'", "'paid'"], dtype=object)

In [63]:
a=a.map(lambda x: 'Paid' if x in np.array(["'paid'", "u'paid'"]) else x)
a.unique()

array(['No', nan, 'Free', 'Paid'], dtype=object)

In [64]:
Rest_new['WiFi']=a

In [65]:
Rest_new['Alcohol'].unique()

array(["u'full_bar'", "u'beer_and_wine'", "u'none'", nan, "'none'",
       "'beer_and_wine'", "'full_bar'", 'None'], dtype=object)

In [66]:
Alc = Rest_new['Alcohol'].map(lambda x: 'Full_Bar' if x in np.array(["u'full_bar'", "'full_bar'"]) else x)
Alc.unique()

array(['Full_Bar', "u'beer_and_wine'", "u'none'", nan, "'none'",
       "'beer_and_wine'", 'None'], dtype=object)

In [67]:
Alc = Alc.map(lambda x: 'Beer&Wine' if x in np.array(["u'beer_and_wine'", "'beer_and_wine'"]) else x)
Alc.unique()

array(['Full_Bar', 'Beer&Wine', "u'none'", nan, "'none'", 'None'],
      dtype=object)

In [68]:
Alc = Alc.map(lambda x: 'No' if x in np.array(["u'none'", "'none'",'None']) else x)
Alc.unique()

array(['Full_Bar', 'Beer&Wine', 'No', nan], dtype=object)

###### Cleaned Version:

In [69]:
Rest_new['Alcohol'] = Alc
Rest_new.head()

,name,business_id,address,cuisine,postal_code,hours,latitude,longitude,review_count,stars,OutdoorSeating,BusinessAcceptsCreditCards,RestaurantsDelivery,RestaurantsReservations,WiFi,Alcohol,categories,city
1,Emerald Chinese Restaurant,QXAEGFB4oINsVuTFxEYKFQ,30 Eglinton Avenue W,Chinese,L5R 3E7,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",43.605499,-79.652289,128,2.5,False,NaN,False,True,No,Full_Bar,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga
2,Musashi Japanese Restaurant,gnKjwL_1w79qoiV3IC_xQQ,"10110 Johnston Rd, Ste 15",Japanese,28210,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",35.092564,-80.859132,170,4.0,False,True,False,True,No,Beer&Wine,"Sushi Bars, Restaurants, Japanese",Charlotte
3,Taco Bell,1Dfx3zM-rW4n-31KeC8sJg,2450 E Indian School Rd,Mexican,85016,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",33.495194,-112.028588,18,3.0,False,True,False,False,No,No,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix
4,Marco's Pizza,fweCYi8FmbJXHCqLnwuk8w,5981 Andrews Rd,Italian,44060,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",41.708520,-81.359556,16,4.0,False,True,True,False,NaN,No,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake
5,Carluccio's Tivoli Gardens,PZ-LZzSlhSe9utkQYU8pFg,"1775 E Tropicana Ave, Ste 29",Italian,89119,NaN,36.100016,-115.128529,40,4.0,False,True,False,True,No,Full_Bar,"Restaurants, Italian",Las Vegas


### Splitting up restaurant hours:

Summary of Actions:

- Clean up hours to split into multiple columns regarding to open and close time of each day.
- Check if every restaurant open and close once per day.
- Use the defined function to split keys(days) and values (hours) of hours dictionary for later information extraction.

In [70]:
print(Rest_new['hours'][Rest_new['hours'].notnull()].map(lambda x: x.values()).map(len).sort_values().value_counts())

7    34043
6     8181
5     2632
4      233
3      128
2       89
1       70
Name: hours, dtype: int64


In [71]:
def merge(x,y):
    result = []
    try:
        for i in x:
            index = x.index(i)
            result.append(i)
            result.append(y[index])
        return result
    except TypeError:
        result = [np.NaN, np.NaN]

In [72]:
Rest_new['business_days']=Rest_new['hours'][Rest_new['hours'].notnull()].map(lambda x:list(x.keys()))
Rest_new['business_hours']=Rest_new['hours'][Rest_new['hours'].notnull()].map(lambda x:list(x.values()))
Rest_new['hours_day'] = Rest_new.apply(lambda row: merge(row['business_days'], row['business_hours']), axis=1)

In [73]:
Rest_new_hours = Rest_new[:]
Rest_new_hours.head(10)

,name,business_id,address,cuisine,postal_code,hours,latitude,longitude,review_count,stars,...,BusinessAcceptsCreditCards,RestaurantsDelivery,RestaurantsReservations,WiFi,Alcohol,categories,city,business_days,business_hours,hours_day
1,Emerald Chinese Restaurant,QXAEGFB4oINsVuTFxEYKFQ,30 Eglinton Avenue W,Chinese,L5R 3E7,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",43.605499,-79.652289,128,2.5,...,NaN,False,True,No,Full_Bar,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"[Monday, Tuesday, Wednesday, Thursday, Friday,...","[9:0-0:0, 9:0-0:0, 9:0-0:0, 9:0-0:0, 9:0-1:0, ...","[Monday, 9:0-0:0, Tuesday, 9:0-0:0, Wednesday,..."
2,Musashi Japanese Restaurant,gnKjwL_1w79qoiV3IC_xQQ,"10110 Johnston Rd, Ste 15",Japanese,28210,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",35.092564,-80.859132,170,4.0,...,True,False,True,No,Beer&Wine,"Sushi Bars, Restaurants, Japanese",Charlotte,"[Monday, Wednesday, Thursday, Friday, Saturday...","[17:30-21:30, 17:30-21:30, 17:30-21:30, 17:30-...","[Monday, 17:30-21:30, Wednesday, 17:30-21:30, ..."
3,Taco Bell,1Dfx3zM-rW4n-31KeC8sJg,2450 E Indian School Rd,Mexican,85016,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",33.495194,-112.028588,18,3.0,...,True,False,False,No,No,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"[Monday, Tuesday, Wednesday, Thursday, Friday,...","[7:0-0:0, 7:0-0:0, 7:0-0:0, 7:0-1:0, 7:0-1:0, ...","[Monday, 7:0-0:0, Tuesday, 7:0-0:0, Wednesday,..."
4,Marco's Pizza,fweCYi8FmbJXHCqLnwuk8w,5981 Andrews Rd,Italian,44060,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",41.708520,-81.359556,16,4.0,...,True,True,False,NaN,No,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake,"[Monday, Tuesday, Wednesday, Thursday, Friday,...","[10:0-0:0, 10:0-0:0, 10:0-0:0, 10:0-0:0, 10:0-...","[Monday, 10:0-0:0, Tuesday, 10:0-0:0, Wednesda..."
5,Carluccio's Tivoli Gardens,PZ-LZzSlhSe9utkQYU8pFg,"1775 E Tropicana Ave, Ste 29",Italian,89119,NaN,36.100016,-115.128529,40,4.0,...,True,False,True,No,Full_Bar,"Restaurants, Italian",Las Vegas,NaN,NaN,None
6,Marathon Diner,1RHY4K3BD22FK7Cfftn8Mg,"Center Core - Food Court, Fl 3, Pittsburgh Int...",Others,15231,NaN,40.496177,-80.246011,35,4.0,...,True,False,False,NaN,NaN,"Sandwiches, Salad, Restaurants, Burgers, Comfo...",Pittsburgh,NaN,NaN,None
7,Maria's Mexican Restaurant & Bakery,tstimHoMcYbkSC4eBA1wEg,6055 E Lake Mead Blvd,Mexican,89156,"{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0'...",36.195615,-115.040529,184,4.5,...,True,False,False,No,Beer&Wine,"Mexican, Restaurants, Patisserie/Cake Shop, Fo...",Las Vegas,"[Monday, Tuesday, Wednesday, Thursday, Friday,...","[11:0-21:0, 10:0-21:0, 10:0-21:0, 10:0-21:0, 1...","[Monday, 11:0-21:0, Tuesday, 10:0-21:0, Wednes..."
8,Bolt Fresh Bar,NDuUMJfrWk52RA-H-OtrpA,1170 Queen Street W,Others,M6J 1J5,"{'Monday': '8:0-21:0', 'Tuesday': '8:0-21:0', ...",43.642889,-79.425429,57,3.0,...,NaN,NaN,NaN,No,NaN,"Juice Bars & Smoothies, Food, Restaurants, Fas...",Toronto,"[Monday, Tuesday, Wednesday, Thursday, Friday,...","[8:0-21:0, 8:0-21:0, 8:0-21:0, 8:0-21:0, 8:0-2...","[Monday, 8:0-21:0, Tuesday, 8:0-21:0, Wednesda..."
9,The Steady Cafe & Bar,SP_YXIEwkFPPl_9anCYmpQ,1051 Bloor Street W,Others,M6H 1M4,"{'Tuesday': '9:0-18:0', 'Wednesday': '9:0-18:0...",43.660494,-79.432099,29,3.5,...,NaN,False,True,Free,Full_Bar,"Restaurants, Nightlife, Breakfast & Brunch, Ve...",Toronto,"[Tuesday, Wednesday, Thursday, Friday, Saturda...","[9:0-18:0, 9:0-18:0, 9:0-18:0, 9:0-2:0, 10:0-2...","[Tuesday, 9:0-18:0, Wednesday, 9:0-18:0, Thurs..."
10,Manzetti's Tavern,BvYU3jvGd0TJ7IyZdfiN2Q,6401 Morrison Blvd,"Italian, American",28211,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",35.156338,-80.831878,16,3.5,...,True,False,True,NaN,Full_Bar,"Sandwiches, Italian, American (Traditional), A...",Charlotte,"[Monday, Tuesday, Wednesday, Thursday, Friday,...","[11:0-22:0, 11:0-22:0, 11:0-23:0, 11:0-23:0, 1...","[Monday, 11:0-22:0, Tuesday, 11:0-22:0, Wednes..."


In [74]:
Rest_new_hours['hours_day'][Rest_new_hours['hours_day'].notnull()] = Rest_new_hours['hours_day'][Rest_new['hours_day'].notnull()].map(lambda x: ''.join(x))
Rest_new_hours.head()

,name,business_id,address,cuisine,postal_code,hours,latitude,longitude,review_count,stars,...,BusinessAcceptsCreditCards,RestaurantsDelivery,RestaurantsReservations,WiFi,Alcohol,categories,city,business_days,business_hours,hours_day
1,Emerald Chinese Restaurant,QXAEGFB4oINsVuTFxEYKFQ,30 Eglinton Avenue W,Chinese,L5R 3E7,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",43.605499,-79.652289,128,2.5,...,NaN,False,True,No,Full_Bar,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"[Monday, Tuesday, Wednesday, Thursday, Friday,...","[9:0-0:0, 9:0-0:0, 9:0-0:0, 9:0-0:0, 9:0-1:0, ...",Monday9:0-0:0Tuesday9:0-0:0Wednesday9:0-0:0Thu...
2,Musashi Japanese Restaurant,gnKjwL_1w79qoiV3IC_xQQ,"10110 Johnston Rd, Ste 15",Japanese,28210,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",35.092564,-80.859132,170,4.0,...,True,False,True,No,Beer&Wine,"Sushi Bars, Restaurants, Japanese",Charlotte,"[Monday, Wednesday, Thursday, Friday, Saturday...","[17:30-21:30, 17:30-21:30, 17:30-21:30, 17:30-...",Monday17:30-21:30Wednesday17:30-21:30Thursday1...
3,Taco Bell,1Dfx3zM-rW4n-31KeC8sJg,2450 E Indian School Rd,Mexican,85016,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",33.495194,-112.028588,18,3.0,...,True,False,False,No,No,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"[Monday, Tuesday, Wednesday, Thursday, Friday,...","[7:0-0:0, 7:0-0:0, 7:0-0:0, 7:0-1:0, 7:0-1:0, ...",Monday7:0-0:0Tuesday7:0-0:0Wednesday7:0-0:0Thu...
4,Marco's Pizza,fweCYi8FmbJXHCqLnwuk8w,5981 Andrews Rd,Italian,44060,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",41.708520,-81.359556,16,4.0,...,True,True,False,NaN,No,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake,"[Monday, Tuesday, Wednesday, Thursday, Friday,...","[10:0-0:0, 10:0-0:0, 10:0-0:0, 10:0-0:0, 10:0-...",Monday10:0-0:0Tuesday10:0-0:0Wednesday10:0-0:0...
5,Carluccio's Tivoli Gardens,PZ-LZzSlhSe9utkQYU8pFg,"1775 E Tropicana Ave, Ste 29",Italian,89119,NaN,36.100016,-115.128529,40,4.0,...,True,False,True,No,Full_Bar,"Restaurants, Italian",Las Vegas,NaN,NaN,None


In [75]:
Rest_new_hours['Monday_Open']=Rest_new_hours['hours_day'].str.extract('[M][o][n][d][a][y](\d*[:]\d*)[-]\d*[:]\d*')
Rest_new_hours['Tuesday_Open']=Rest_new_hours['hours_day'].str.extract('[T][u][e][s][d][a][y](\d*[:]\d*)[-]\d*[:]\d*')
Rest_new_hours['Wednesday_Open']=Rest_new_hours['hours_day'].str.extract('[W][e][d][n][e][s][d][a][y](\d*[:]\d*)[-]\d*[:]\d*')
Rest_new_hours['Thursday_Open']=Rest_new_hours['hours_day'].str.extract('[T][h][u][r][s][d][a][y](\d*[:]\d*)[-]\d*[:]\d*')
Rest_new_hours['Friday_Open']=Rest_new_hours['hours_day'].str.extract('[F][r][i][d][a][y](\d*[:]\d*)[-]\d*[:]\d*')
Rest_new_hours['Saturday_Open']=Rest_new_hours['hours_day'].str.extract('[S][a][t][u][r][d][a][y](\d*[:]\d*)[-]\d*[:]\d*')
Rest_new_hours['Sunday_Open']=Rest_new_hours['hours_day'].str.extract('[S][u][n][d][a][y](\d*[:]\d*)[-]\d*[:]\d*')
Rest_new_hours['Monday_Close']=Rest_new_hours['hours_day'].str.extract('[M][o][n][d][a][y]\d*[:]\d*[-](\d*[:]\d*)')
Rest_new_hours['Tuesday_Close']=Rest_new_hours['hours_day'].str.extract('[T][u][e][s][d][a][y]\d*[:]\d*[-](\d*[:]\d*)')
Rest_new_hours['Wednesday_Close']=Rest_new_hours['hours_day'].str.extract('[[W][e][d][n][e][s][d][a][y]\d*[:]\d*[-](\d*[:]\d*)')
Rest_new_hours['Thursday_Close']=Rest_new_hours['hours_day'].str.extract('[T][h][u][r][s][d][a][y]\d*[:]\d*[-](\d*[:]\d*)')
Rest_new_hours['Friday_Close']=Rest_new_hours['hours_day'].str.extract('[F][r][i][d][a][y]\d*[:]\d*[-](\d*[:]\d*)')
Rest_new_hours['Saturday_Close']=Rest_new_hours['hours_day'].str.extract('[S][a][t][u][r][d][a][y]\d*[:]\d*[-](\d*[:]\d*)')
Rest_new_hours['Sunday_Close']=Rest_new_hours['hours_day'].str.extract('[S][u][n][d][a][y]\d*[:]\d*[-](\d*[:]\d*)')

In [76]:
Rest_new_hours.head(5)

,name,business_id,address,cuisine,postal_code,hours,latitude,longitude,review_count,stars,...,Friday_Open,Saturday_Open,Sunday_Open,Monday_Close,Tuesday_Close,Wednesday_Close,Thursday_Close,Friday_Close,Saturday_Close,Sunday_Close
1,Emerald Chinese Restaurant,QXAEGFB4oINsVuTFxEYKFQ,30 Eglinton Avenue W,Chinese,L5R 3E7,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",43.605499,-79.652289,128,2.5,...,9:0,9:0,9:0,0:0,0:0,0:0,0:0,1:0,1:0,0:0
2,Musashi Japanese Restaurant,gnKjwL_1w79qoiV3IC_xQQ,"10110 Johnston Rd, Ste 15",Japanese,28210,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",35.092564,-80.859132,170,4.0,...,17:30,17:30,17:30,21:30,NaN,21:30,21:30,22:0,22:0,21:0
3,Taco Bell,1Dfx3zM-rW4n-31KeC8sJg,2450 E Indian School Rd,Mexican,85016,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",33.495194,-112.028588,18,3.0,...,7:0,7:0,7:0,0:0,0:0,0:0,1:0,1:0,1:0,0:0
4,Marco's Pizza,fweCYi8FmbJXHCqLnwuk8w,5981 Andrews Rd,Italian,44060,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",41.708520,-81.359556,16,4.0,...,10:0,10:0,10:0,0:0,0:0,0:0,0:0,1:0,1:0,0:0
5,Carluccio's Tivoli Gardens,PZ-LZzSlhSe9utkQYU8pFg,"1775 E Tropicana Ave, Ste 29",Italian,89119,NaN,36.100016,-115.128529,40,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
Rest_new_hours.drop(['hours_day','business_days','business_hours'],axis=1,inplace=True)
Rest_new_hours.columns

Index(['name', 'business_id', 'address', 'cuisine', 'postal_code', 'hours',
       'latitude', 'longitude', 'review_count', 'stars', 'OutdoorSeating',
       'BusinessAcceptsCreditCards', 'RestaurantsDelivery',
       'RestaurantsReservations', 'WiFi', 'Alcohol', 'categories', 'city',
       'Monday_Open', 'Tuesday_Open', 'Wednesday_Open', 'Thursday_Open',
       'Friday_Open', 'Saturday_Open', 'Sunday_Open', 'Monday_Close',
       'Tuesday_Close', 'Wednesday_Close', 'Thursday_Close', 'Friday_Close',
       'Saturday_Close', 'Sunday_Close'],
      dtype='object')

In [78]:
def str2time(val):
    try:
        return dt.datetime.strptime(val, '%H:%M').time()
    except:
        return pd.NaT

In [79]:
#Select the featurings concerning the opening and closing of the days of the week
Rest_new_hours.iloc[:,18:32]=Rest_new_hours.iloc[:,18:32].astype(str)
Rest_new_hours.iloc[:,18:32]=Rest_new_hours.iloc[:,18:32].applymap(lambda x: str2time(x))
Rest_new_hours.iloc[:,18:32].head()

,Monday_Open,Tuesday_Open,Wednesday_Open,Thursday_Open,Friday_Open,Saturday_Open,Sunday_Open,Monday_Close,Tuesday_Close,Wednesday_Close,Thursday_Close,Friday_Close,Saturday_Close,Sunday_Close
1,09:00:00,09:00:00,09:00:00,09:00:00,09:00:00,09:00:00,09:00:00,00:00:00,00:00:00,00:00:00,00:00:00,01:00:00,01:00:00,00:00:00
2,17:30:00,NaT,17:30:00,17:30:00,17:30:00,17:30:00,17:30:00,21:30:00,NaT,21:30:00,21:30:00,22:00:00,22:00:00,21:00:00
3,07:00:00,07:00:00,07:00:00,07:00:00,07:00:00,07:00:00,07:00:00,00:00:00,00:00:00,00:00:00,01:00:00,01:00:00,01:00:00,00:00:00
4,10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,00:00:00,00:00:00,00:00:00,00:00:00,01:00:00,01:00:00,00:00:00
5,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [80]:
Rest_new_hours.loc[3801]

name                                                                Chang Jiang
business_id                                              asAHqgKQtCWiBbC3XriSIQ
address                                                         646 S Gammon Rd
cuisine                                                                 Chinese
postal_code                                                               53719
hours                         {'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...
latitude                                                                43.0505
longitude                                                               -89.504
review_count                                                                 31
stars                                                                       3.5
OutdoorSeating                                                            False
BusinessAcceptsCreditCards                                                 True
RestaurantsDelivery                     

In [81]:
Rest_new_hours.drop('hours',axis=1,inplace=True)
Rest_new_hours.head()

,name,business_id,address,cuisine,postal_code,latitude,longitude,review_count,stars,OutdoorSeating,...,Friday_Open,Saturday_Open,Sunday_Open,Monday_Close,Tuesday_Close,Wednesday_Close,Thursday_Close,Friday_Close,Saturday_Close,Sunday_Close
1,Emerald Chinese Restaurant,QXAEGFB4oINsVuTFxEYKFQ,30 Eglinton Avenue W,Chinese,L5R 3E7,43.605499,-79.652289,128,2.5,False,...,09:00:00,09:00:00,09:00:00,00:00:00,00:00:00,00:00:00,00:00:00,01:00:00,01:00:00,00:00:00
2,Musashi Japanese Restaurant,gnKjwL_1w79qoiV3IC_xQQ,"10110 Johnston Rd, Ste 15",Japanese,28210,35.092564,-80.859132,170,4.0,False,...,17:30:00,17:30:00,17:30:00,21:30:00,NaT,21:30:00,21:30:00,22:00:00,22:00:00,21:00:00
3,Taco Bell,1Dfx3zM-rW4n-31KeC8sJg,2450 E Indian School Rd,Mexican,85016,33.495194,-112.028588,18,3.0,False,...,07:00:00,07:00:00,07:00:00,00:00:00,00:00:00,00:00:00,01:00:00,01:00:00,01:00:00,00:00:00
4,Marco's Pizza,fweCYi8FmbJXHCqLnwuk8w,5981 Andrews Rd,Italian,44060,41.708520,-81.359556,16,4.0,False,...,10:00:00,10:00:00,10:00:00,00:00:00,00:00:00,00:00:00,00:00:00,01:00:00,01:00:00,00:00:00
5,Carluccio's Tivoli Gardens,PZ-LZzSlhSe9utkQYU8pFg,"1775 E Tropicana Ave, Ste 29",Italian,89119,36.100016,-115.128529,40,4.0,False,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


### Saving dataframe

Summary of Actions:

- Saving dataframe in pickle file

In [82]:
#Saving the dataset in pickle
Rest_new_hours.to_pickle('../dataset/restaurants.pickle') 

### Cleaning of the Review Dataset:

#### Working with Review pickle file 



Summary of Actions:
- Reset the index to 1 - for ease of reading
- Rearranging the columns in the dataframe
- Updating the timestamp to include only the date format (YYYY-MM-DD).
- We used the pandas to_datetime to drop the time of the 'date'column
- Save the pickle file again with the new operations performed.

In [83]:
pickle_review = open("../dataset/all_review.pickle","rb")
review = pickle.load(pickle_review)
review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
5,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,4,0,0,0,I'll be the first to admit that I was not exci...,2013-01-20 13:25:59
6,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3,5,4,5,Tracy dessert had a big name in Hong Kong and ...,2016-05-07 01:21:02
7,8e9HxxLjjqc9ez5ezzN7iQ,d6xvYpyzcfbF_AZ8vMB7QA,zvO-PJCpNk4fgAVUnExYAA,1,3,1,1,This place has gone down hill. Clearly they h...,2010-10-05 19:12:35
10,kbtscdyz6lvrtGjD1quQTg,FIk4lQQu1eTe2EpzQ4xhBA,8mIrX_LrOnAqWsB5JrOojQ,4,0,0,0,"Like walking back in time, every Saturday morn...",2011-11-30 02:11:15


In [84]:
Review = review.reset_index(drop = True)
Review.index +=1
Review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
1,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
2,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,4,0,0,0,I'll be the first to admit that I was not exci...,2013-01-20 13:25:59
3,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3,5,4,5,Tracy dessert had a big name in Hong Kong and ...,2016-05-07 01:21:02
4,8e9HxxLjjqc9ez5ezzN7iQ,d6xvYpyzcfbF_AZ8vMB7QA,zvO-PJCpNk4fgAVUnExYAA,1,3,1,1,This place has gone down hill. Clearly they h...,2010-10-05 19:12:35
5,kbtscdyz6lvrtGjD1quQTg,FIk4lQQu1eTe2EpzQ4xhBA,8mIrX_LrOnAqWsB5JrOojQ,4,0,0,0,"Like walking back in time, every Saturday morn...",2011-11-30 02:11:15


In [85]:
Review = Review[['business_id', 'user_id', 'review_id', 'date', 'cool','funny','useful','stars']]
Review.head()

,business_id,user_id,review_id,date,cool,funny,useful,stars
1,ikCg8xy5JIg_NGPx-MSIDA,dacAIZ6fTM6mqwW5uxkskg,yi0R0Ugj_xUx_Nek0-_Qig,2018-01-09 20:56:38,0,0,0,5
2,eU_713ec6fTGNO4BegRaww,w31MKYsNFMrjhWxxAb5wIw,fdiNeiN_hoCxCMy2wTRW9g,2013-01-20 13:25:59,0,0,0,4
3,3fw2X5bZYeW9xCz_zGhOHg,jlu4CztcSxrKx56ba1a5AQ,G7XHMxG0bx9oBJNECG4IFg,2016-05-07 01:21:02,5,4,5,3
4,zvO-PJCpNk4fgAVUnExYAA,d6xvYpyzcfbF_AZ8vMB7QA,8e9HxxLjjqc9ez5ezzN7iQ,2010-10-05 19:12:35,1,1,3,1
5,8mIrX_LrOnAqWsB5JrOojQ,FIk4lQQu1eTe2EpzQ4xhBA,kbtscdyz6lvrtGjD1quQTg,2011-11-30 02:11:15,0,0,0,4


In [86]:
#Saving the dataset in pickle 
review.to_pickle('../dataset/all_review.pickle')

### Cleaning Users Dataset

#### Working with users pickle file

Summary of Actions:
- After processing the data, we have shrunk the dataset from 22 columns to 11 columns. The 'compliment' columns are all dropped because they function very similar to 'cool' and 'funny' columns which are also counting how many different kinds of compliments the user got from others. So, to remove the redundancy, we have eliminated those variables.
- Since we have extracted all data, the index was not in order. Therefore, the first step is to reset the index and make the first index '1'. 
- Second, we re-arrange the columns order so the most important information will be shown first which makes it easier for readers to gain insights from the data frame.  
- Third, the 'yelping_since' included data and time (hour and minute) which we do not need 'time' for our analysis. Therefore, we used pandas to_datatime function to drop the 'time' in that column.
- After that, we worked on the multivalued columns: elite and friends. 'elite' columns contained all the years that the user was a elite member in a string format. 
- We decided that having the year details do not help with analyzing the dataset, instead, counting how many years the user is a elite member provides more useful information. 
- Therefore, we first used regular expression to find all the years which would also convert strings to lists.
- The similar methods apply to 'friends' too, but  instead of regular expression, we used a string method to split the strings. 
- Consequently, we changed 'name' to 'user_name' to specify which dataset this column belongs to.
- Save the pickle file again with the new operations performed.

In [87]:
pickle_users = open("../dataset/all_users.pickle","rb")
users = pickle.load(pickle_users)

In [88]:
#dropping org index 
users = users.reset_index(drop=True)
users.index +=1

In [89]:
titles = ['user_id','name','average_stars','yelping_since','review_count','elite','fans','useful','cool','funny','friends']
users =users.reindex(columns=titles)

#rename columns
users = users.rename(columns={'name':'user_name','review_count':'review'})   

In [90]:
#converting timestamp to date 
users['yelping_since'] = pd.to_datetime(users['yelping_since'])
users['yelping_since'] = users['yelping_since'].dt.date

In [91]:
users['elite'] = users['elite'].apply(lambda x: re.findall('20\d\d',x))

In [92]:
users['elite'] = users['elite'].apply(lambda x: len(x))

In [93]:
users['friends'].str.split(',')
users['friends'] = users['friends'].apply(lambda x: len(x))

In [94]:
users = users.rename(columns={'elite':'years_of_elite'})
users.head()

,user_id,user_name,average_stars,yelping_since,review,years_of_elite,fans,useful,cool,funny,friends
1,l6BmjZMeQD3rDxWUbiAiow,Rashmi,4.03,2013-10-08,95,3,5,84,25,17,2374
2,4XChL029mKr5hydo79Ljxg,Jenna,3.63,2013-02-21,33,0,4,48,16,22,27646
3,bc8C_eETBWL0olvFSJJd0w,David,3.71,2013-10-04,16,0,0,28,10,8,358
4,dD0gZpBctWGdWo9WlGuhlA,Angela,4.85,2014-05-22,17,0,5,30,14,4,12598
5,MM4RJAeH6yuaN8oZDSt0RA,Nancy,4.08,2013-10-23,361,4,39,1114,665,279,5542


In [95]:
users.to_pickle('../dataset/all_users.pickle')

### Cleaning Tip Dataset

#### Working with tip pickle file

Summary of Actions:

- Since the original tip dataset only contain business_id, we extracted 'business_id' and 'name' from restaurant dataset in order to add the 'name' column in the tip_new dataset. 
- We added the column through doing an inner join.
- Also, we used the pandas to_datetime to drop the time of the 'date'column. 
- After that, we rearrange the column orders and renamed the columns names so we do not have same names accross different dataset.
- Save the pickle file again with the new operations performed.

In [96]:
pickle_tip = open("../dataset/all_tips.pickle","rb")
tip = pickle.load(pickle_tip)
tip = tip.set_index(keys = 'business_id')

In [97]:
#load in restaurant pickle file in order to get the restaurant names
pickle_restaurant = open("../dataset/restaurants.pickle","rb")
restaurant = pickle.load(pickle_restaurant)
restaurant_new = restaurant[['name', 'business_id']]
restaurant_new = restaurant_new.set_index(keys = 'business_id')

In [98]:
tip_new = tip.join(restaurant_new, how = 'inner')

In [99]:
tip_new['date'] = pd.to_datetime(tip_new['date'])
tip_new['date'] = tip_new['date'].dt.date
tip_new.drop(columns = ['text']) 

,user_id,date,compliment_count,name
business_id,,,,
--1UhMGODdWsrMastO9DZw,4YHZzwlxEMi7zWO6Osszdw,2017-07-26,0,The Spicy Amigos
--6MefnULPED_I942VcFNA,EiP1OFgs-XGcKZux0OKWIA,2013-03-24,0,John's Chinese BBQ Restaurant
--6MefnULPED_I942VcFNA,2oMkzQcRL7-d7URt3Xo_Xg,2014-07-11,0,John's Chinese BBQ Restaurant
--6MefnULPED_I942VcFNA,Md9s_mb8Dxsl5MbjIp-DjQ,2014-06-16,0,John's Chinese BBQ Restaurant
--6MefnULPED_I942VcFNA,3yMtpQ_wV4ZGg6E69uE1PQ,2015-02-20,0,John's Chinese BBQ Restaurant
--6MefnULPED_I942VcFNA,0cUzu82KJiE5_xZA0Iu3ZQ,2017-01-15,0,John's Chinese BBQ Restaurant
--6MefnULPED_I942VcFNA,mFwRTTDW0Yr-rFkTF2cFsw,2015-01-04,0,John's Chinese BBQ Restaurant
--6MefnULPED_I942VcFNA,RX9MH2jHaK90KF-2rU9Mvg,2017-11-05,0,John's Chinese BBQ Restaurant
--6MefnULPED_I942VcFNA,6tbXpUIU6upoeqWNDo9k_A,2015-10-12,0,John's Chinese BBQ Restaurant


In [100]:
titles = ['name', 'date', 'user_id']
tip_new = tip_new.reindex(columns=titles)

In [101]:
tip_new = tip_new.rename(columns={'name':'restaurant_name', 'date':'tips_date'})

In [102]:
tip_new.to_pickle('../dataset/all_tips.pickle')